In [1]:
# Pythonの基本ライブラリ
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# ファイル操作
import os
import glob

# Jupyter上にHTMLを表示する
from IPython.display import HTML

# 日付の変換
import datetime

In [2]:
def output_html(video_ids, titles):
    html = '<div style="float:left;">'
    for i, video_id in enumerate(video_ids):
        html += ('<img src="http://img.youtube.com/vi/'+video_id+'/sddefault.jpg "alt="取得できませんでした" width="80">')
        url = "https://www.youtube.com/watch?v="+video_id
        html += ('  <a href="'+url+'">'+titles[i]+'</a>'+'<br>') 
    html += '</div>'
    return html

In [9]:
# 設定値の変更
current_year = 2022
current_month = 8

In [10]:
# 全動画から今月の動画を取得
file_names = glob.glob('../dataset/output/hololive/*')
for i, file_name in enumerate(file_names):
    videos = pd.read_csv(file_name)
    dates = videos['Date']

    subjects = np.zeros(len(dates), dtype=bool)
    for j, date in enumerate(dates):
        tdatetime = datetime.datetime.strptime(date, '%Y-%m-%dT%H:%M:%SZ')
        tdate = datetime.date(tdatetime.year, tdatetime.month, tdatetime.day)
        tdate_start = datetime.date(current_year, current_month, 1)
        tdate_end = datetime.date(current_year, current_month+1, 1)

        if tdate >= tdate_start and tdate < tdate_end:
            subjects[j] = True
    
    videos['Subject'] = subjects
    filtered_videos = videos[videos['Subject'] == True]
    
    # 差分を追加する場合
    if i == 0:
        all_videos = filtered_videos
    else:
        all_videos = pd.concat([all_videos, filtered_videos])

# 今月の動画のタイトルを取得
titles = all_videos['Title']
durations = all_videos['Duration']
subjects = np.zeros(len(titles), dtype=bool)

keywords = ['MV', '3D', 'ライブ', 'LIVE', '重大', '発表', '歌', 'オリジナル', '周年', '記念', 'Anniversary', 'singing']
for i, title in enumerate(titles):
    title_replaced = title.replace('ホロライブ', '')
    title_replaced = title_replaced.replace('HOLOLIVE', '')
    # キアライブも多いので対象から除外する
    title_replaced = title_replaced.replace('キアライブ', '')
    for keyword in keywords:
        if keyword in title_replaced:
            subjects[i] = True

# 音楽っぽい動画は優先的に取り出す
for i, duration in enumerate(durations):
    if duration > 90 and duration < 480:
        subjects[i] = True

all_videos['Subject'] = subjects
filtered_videos = all_videos[all_videos['Subject'] == True]
filtered_videos.head(1)

filtered_videos_sorted = filtered_videos.sort_values('ViewCount', ascending=False)
video_ids = np.array(filtered_videos_sorted['Id'])
titles = np.array(filtered_videos_sorted['Title'])
n = 10000 # 現状全ての動画を表示するようにしている
HTML(output_html(video_ids[:n], titles[:n]))